In [ ]:
import psycopg2
import pandas as pd
import os
import itertools
import numpy as np

conn = psycopg2.connect(
    host='localhost',
    port=54340,
    dbname='salon24',
    password='pass',
    user='postgres',
    )
conn.autocommit = False

In [ ]:
import datetime
from datetime import timedelta 

authors = "authors"
comments = "comments"
posts = "posts"



start_date_comments = datetime.date(2008, 1, 1)
end_date_comments = datetime.date(2013,7, 6)

In [ ]:
df = pd.read_csv("FeatureTable_SALON.csv", sep=";")

In [ ]:
def get_all_users():
    cur = conn.cursor()
    query = """
    SELECT id, name FROM {} order by id
    """.format(authors)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
def get_all_ids():
    result = get_all_users()
    return [x[0] for x in result]

In [ ]:
def save_data_to_file(folder_name, file_name, data):
    try:
        # Create target Directory
        os.mkdir(folder_name)
        print("Directory " , folder_name ,  " Created ") 
    except FileExistsError:
        e = 1
        #print("Directory " , folder_name ,  " already exists")
    data.to_csv(folder_name + "/" + file_name, index=False)

In [ ]:
def clear_none_values(results):
    data = []
    for res in results:
        res = [0 if v is None else v for v in res]
        data.append(res)
    return data

In [ ]:
def generate_feature(start_date, end_date, is_clear, feature_function, columns, folder_name):
    
    while start_date < end_date:
        print(start_date)
        results = feature_function(start_date, start_date + timedelta(days=28))
        res = []
        if is_clear:
            res = results
        else:
            res = clear_none_values(results)
        data = []
        data.extend(res)
        df = pd.DataFrame(data, columns = columns )
        df['start_date'] = pd.Series(start_date, index=df.index)
        df['end_date'] = pd.Series(end_date, index=df.index)
        save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", df)
        start_date += timedelta(days=14)

In [ ]:
def get_number_of_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT author_id, COUNT(*) FROM {}
    WHERE date between '{}' and '{}'
    GROUP BY author_id
    ORDER BY author_id
    """.format(posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
#"NumberOfUsersPostsWithoutZeros"
folder = "SALON_NumberOfUserPosts"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, True, get_number_of_users_posts, columns, folder)

In [ ]:
def get_number_of_written_comments_to_other_users(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT c.author_id, COUNT(*) FROM {} c
    JOIN {} p ON p.id = c.post_id AND c.author_id <> p.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY c.author_id
    ORDER BY c.author_id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "SALON_NumberOfCommentsToOtherUsers"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, True, get_number_of_written_comments_to_other_users, columns, folder)

In [ ]:
def get_number_of_received_responses_to_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT p.author_id as id, COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id AND p.author_id <> c.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "SALON_NumberOfReceivedResponsesToPost"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_responses_to_users_posts, columns, folder)

In [ ]:
def get_number_of_received_unique_users_responses_to_users_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT p.author_id as id, COUNT(c.author_id ) as resp_amount 
    FROM {} c JOIN {} p ON p.id = c.post_id 
    AND p.author_id <> c.author_id 
    WHERE c.date between '{}' AND '{}' 
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "SALON_NumberOfReceivedUniqueUsersResponsesToPost"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_unique_users_responses_to_users_posts, columns, folder)

In [ ]:
def get_number_of_all_responses_from_unique_users_in_slot(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT p.author_id as id, COUNT(c.author_id ) as resp_amount 
    FROM {} c JOIN {} p ON p.id = c.post_id 
    AND p.author_id <> c.author_id 
    WHERE c.date between '{}' AND '{}' 
    GROUP BY p.author_id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "SALON_NumberOfAllResponsesFromUniqueUsersInSlot"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_all_responses_from_unique_users_in_slot, columns, folder)

In [ ]:
def get_number_of_comments_written_by_user_under_his_own_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.resp_amount), stddev(cs.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.resp_amount),
    MAX(cs.resp_amount), MIN(cs.resp_amount) FROM
    (SELECT p.author_id as id, COUNT(*) as resp_amount FROM {} c
    JOIN {} p ON p.id = c.post_id AND p.author_id = c.author_id
    WHERE c.date between '{}' and '{}'
    GROUP BY p.id, p.author_id) as cs
    GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "SALON_NumberOfOwnResponsesToPost"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_comments_written_by_user_under_his_own_posts, columns, folder)

In [ ]:
def get_number_of_received_responses_under_users_comments(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT aggregate.id, AVG(aggregate.resp_amount), stddev(aggregate.resp_amount),
    percentile_cont(0.5) WITHIN GROUP (ORDER BY aggregate.resp_amount),
    percentile_cont(0.75) WITHIN GROUP (ORDER BY aggregate.resp_amount),
    MAX(aggregate.resp_amount), MIN(aggregate.resp_amount) FROM
    (SELECT parent_comments.author_id as id, COUNT(*) as resp_amount FROM {} parent_comments
    JOIN {} children_comments ON parent_comments.id = children_comments.parentcomment_id AND parent_comments.author_id <> children_comments.author_id
    WHERE children_comments.date between '{}' and '{}'
    GROUP BY parent_comments.id, parent_comments.author_id) as aggregate
    GROUP BY aggregate.id
    """.format(comments, comments, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "SALON_NumberOfReceivedResponsesUnderComments"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_number_of_received_responses_under_users_comments, columns, folder)

In [ ]:
def get_num_of_words_of_comments(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len), 
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT c.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c 
                       JOIN {} p ON p.id = c.post_id and c.author_id <> p.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "SALON_NumberOfWordsInUserComments"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_comments, columns, folder)

In [ ]:
def get_num_of_words_of_posts(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT author_id, AVG(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       stddev(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(content, \'\s\'),1)),
                       MAX(array_length(regexp_split_to_array(content, \'\s\'),1)),
                       MIN(array_length(regexp_split_to_array(content, \'\s\'),1))
                       FROM {} WHERE date between '{}' and '{}'
                       GROUP BY author_id
    """.format(posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "SALON_NumberOfWordsInUserPosts"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_posts, columns, folder)

In [ ]:
def get_num_of_words_of_received_post_responses(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT p.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id AND p.author_id <> c.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "SALON_NumberOfWordsInResponsesToUserPosts"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_received_post_responses, columns, folder)

In [ ]:
def get_num_of_words_of_own_post_responses(start_date, end_date):
    cur = conn.cursor()
    query = """
    SELECT cs.id, AVG(cs.len), stddev(cs.len),
                       percentile_cont(0.5) WITHIN GROUP (ORDER BY cs.len),
                       percentile_cont(0.75) WITHIN GROUP (ORDER BY cs.len),
                       MAX(cs.len), MIN(cs.len) FROM
                       (SELECT p.author_id as id, array_length(regexp_split_to_array(c.content, \'\s\'),1) as len FROM {} c
                       JOIN {} p ON p.id = c.post_id AND p.author_id = c.author_id
                       WHERE c.date between '{}' and '{}') as cs
                       GROUP BY cs.id
    """.format(comments, posts, start_date, end_date)
    cur.execute(query)
    result = cur.fetchall()
    conn.commit()
    cur.close()
    return result

In [ ]:
folder = "SALON_NumberOfWordsInOwnPostResponses"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_num_of_words_of_own_post_responses, columns, folder)

In [ ]:
def q75(x):
    return x.quantile(0.75)

In [ ]:
def generate_frequency_feature(start_date, end_date, feature_frequency_function, columns, final_columns, folder_name):
    
    while start_date < end_date:
        print(start_date)
        temp_end_date = start_date + timedelta(days=28)
        temp_start_date = start_date
        dates = []
        while temp_start_date < temp_end_date:
            results = feature_frequency_function(start_date, temp_end_date, temp_start_date, temp_start_date + timedelta(days=7))
            
            data = []
            data.extend(results)
            df = pd.DataFrame(data, columns = columns )
            df['start_date'] = pd.Series(temp_start_date, index=df.index)
            df['end_date'] = pd.Series(temp_start_date + timedelta(days=7), index=df.index)
            
            dates.append(df)
            temp_start_date += timedelta(days=7)
        res = pd.concat(dates, ignore_index=True)
        if res.size != 0:
            res[columns[1]] = pd.to_numeric(res[columns[1]])
#             quartile = res.groupby(['user_id'])[columns[1]].quantile(0.75)
            stats_df = (res.groupby('user_id')[columns[1]].agg(('mean', 'std','median', q75, 'max', 'min')).reset_index())
#             merged_df = pd.merge(stats_df, quartile, on='user_id')
            final_df = stats_df.fillna(0)
            final_df.columns = final_columns
            save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", final_df)
        else:
            empty = pd.DataFrame(data, columns = final_columns )
            save_data_to_file(folder_name, "feature_" + str(start_date) + ".csv", empty)
        start_date += timedelta(days=14)

In [ ]:
def get_post_frequency(real_starts_date, real_end_date, start_date, end_date, timedelta_days=28):
    cur = conn.cursor()
    query = """
    WITH all_users_in_period AS (
    SELECT author_id FROM {} WHERE date between '{}' and '{}' 
    ),
    week AS (SELECT author_id as id, COUNT(*) as num FROM {} 
    WHERE date between '{}' and '{}' 
    GROUP BY author_id)
    
    SELECT a.author_id, coalesce(w.num, 0) as num 
    FROM all_users_in_period a
    LEFT JOIN week w ON a.author_id = w.id
    """.format(posts, real_starts_date, real_end_date, posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ["user_id", "number_of_posts"]
folder = "SALON_FrequencyOfPosts"
final_columns = ['user_id']
final_columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
# final_columns = ['user_id', 'mean_post_frequency', 'std_post_frequency','median_post_frequency','min_post_frequency', 'max_post_frequency', 'q3_post_frequency']
generate_frequency_feature(start_date_comments, end_date_comments, get_post_frequency, columns, final_columns, folder)

In [ ]:
# frequency of writting comments
def get_comments_frequency(real_starts_date, real_end_date, start_date, end_date, timedelta_days=7):
    cur = conn.cursor()
    query = """
    WITH all_users_in_period AS (
    SELECT author_id FROM {} WHERE date between '{}' and '{}' 
    ),
    week AS (SELECT author_id as id, COUNT(*) as num FROM {} 
    WHERE date between '{}' and '{}' 
    GROUP BY author_id)
    
    SELECT a.author_id, coalesce(w.num, 0) as num 
    FROM all_users_in_period a
    LEFT JOIN week w ON a.author_id = w.id
    """.format(comments, real_starts_date, real_end_date,comments, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
columns = ["user_id", "number_of_comments"]

folder = "SALON_FrequencyOfComments"
final_columns = ['user_id']
final_columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
# final_columns = ['user_id', 'mean_comments_frequency', 'std_comments_frequency','median_comments_frequency','min_comments_frequency', 'max_comments_frequency', 'q3_comments_frequency']
generate_frequency_feature(start_date_comments, end_date_comments, get_comments_frequency, columns, final_columns, folder)

In [ ]:
def get_post_activity_without_zeros(start_date, end_date):
    cur = conn.cursor()
    query = """
    WITH disctinct_months AS (
    SELECT DISTINCT author_id, date::date FROM {} WHERE date between '{}' AND '{}'
    ) 
    SELECT author_id as id, count(*) AS post_activity 
    FROM disctinct_months GROUP BY author_id
    """.format(posts, start_date, end_date)
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
folder = "SALON_PostActivityTime"
columns = ['user_id']
columns.extend(list(df.loc[df["NazwaFolderu"] == folder]["NazwaCechy"]))
generate_feature(start_date_comments, end_date_comments, False, get_post_activity_without_zeros, columns, folder)